In [2]:
import pandas as pd
import numpy as np
import os
import warnings
from datetime import date, timedelta, datetime, time
#allow to see all columns
pd.options.display.max_columns = None
#to ignore warning raised by pyvollib for deep ITM options
warnings.simplefilter(action='ignore', category=UserWarning)

class Combined_file:
    '''
    Creates a pickle file from 2 files - FNO_NIFTY and Only_NIFTY
    It takes in file with basic FNO info and converts it to pickle file withP underlying value and all greeks  
    '''
    def __init__(self, FNO_NIFTY_file_name, Only_NIFTY_file_name, Pickle_file_location):
        self.FNO_NIFTY_file_name = FNO_NIFTY_file_name
        self.Only_NIFTY_file_name = Only_NIFTY_file_name
        self.Pickle_file_location = Pickle_file_location
            
    def create_pickle_file(self):
        
        '''
        Takes input of the 2 files mentioned and generates output with greeks and IVs
        It drops all NaN values where the option is deep ITM
        '''
        #load first file from FNO database
        df_fno = pd.read_csv(self.FNO_NIFTY_file_name, 
                     names = ['Instr_info',
                                  'Date',
                                  'Time',
                                  'Open',
                                  'High',
                                  'Low',
                                  'Close',
                                  'Vol',
                                  'Int'], dtype= 'unicode')

        #Take only NIFTY from here by getting rid of BANKNIFTY
        #df_nifty = df_fno[df_fno.Instr_info.str.contains('BANK') != True].copy() 
        # The above doesnt make the cut as there are other entries like FINNIFTY
        # Hence we have below code
        df_nifty = df_fno[(df_fno.Instr_info.apply(lambda x : x[0:5] == 'NIFTY')) & 
                          (df_fno.Instr_info.str.contains('JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC') == True)].copy()
                            # The second part allows to take only NIFTY with dates and avoids futures 
        df_nifty.reset_index(inplace= True)


        #Convert Strike to integer
        # Using lambda and apply to broadcast across dataframe
        df_nifty['Ticker'] = df_nifty['Instr_info'].apply(lambda x: x.strip().split('.')[0][:5])
        df_nifty['Opt_Type'] = df_nifty['Instr_info'].apply(lambda x: x.strip().split('.')[0][-2:])
        df_nifty['Strike'] = df_nifty['Instr_info'].apply(lambda x: x.strip().split('.')[0][12:-2]).astype(float) # as we need it in int form


        #Converting the dates and times (Time, Date and Expiry_date to workable date formats)
        #For dates and times keep the datetime object as the final one to enable doing arithmetic on it
        df_nifty['Expiry_date'] = df_nifty['Instr_info'].apply(lambda x: datetime.strptime(x.strip().split('.')[0][5:12],'%d%b%y'))
        df_nifty['Date'] = df_nifty['Date'].apply(lambda x : datetime.strptime(x,'%d-%m-%Y'))
        df_nifty['Time'] = df_nifty['Time'].apply(lambda x : datetime.strptime(x,'%H:%M:%S'))

        #Drop instrument info as its not longer required
        df_nifty.drop(['Instr_info'],axis = 1, inplace=True)
        df_nifty.drop(['index'], axis = 1, inplace = True)

        #Get minute by minute NIFTY data so that we have current price of underlying
        
        df_only_nifty = pd.read_csv(self.Only_NIFTY_file_name,
                                    header = None,
                                    names = ['only_nifty_date',
                                            'only_nifty_time',
                                            'only_nifty_open',
                                            'only_nifty_high',
                                            'only_nifty_low',
                                            'only_nifty_close',
                                            'only_nifty_vol',
                                            'only_nifty_int'] , dtype= 'unicode')

        df_only_nifty['only_nifty_date'] = df_only_nifty['only_nifty_date'].apply(lambda x: datetime.strptime(x,'%d/%m/%Y'))
        df_only_nifty['only_nifty_time'] = df_only_nifty['only_nifty_time']+':59'
        df_only_nifty['only_nifty_time'] = df_only_nifty['only_nifty_time'].apply(lambda x: datetime.strptime(x,'%H:%M:%S'))
        # Merged two tables to get the Nifty values along with the FNO data
        df_merged = pd.merge(df_nifty, df_only_nifty, how = 'left', left_on= ['Date','Time'], right_on= ['only_nifty_date', 'only_nifty_time'])
        # need to drop faltu columns
        df_merged.drop(columns=['Open','High','Low',
                                'Ticker','only_nifty_date', 'only_nifty_time',
                               'only_nifty_open','only_nifty_high','only_nifty_low'], inplace=True)

        #Generating implied vol and greeks
        import py_vollib.black_scholes_merton.implied_volatility
        import py_vollib_vectorized
        # price of option, S - underlying price, K - strike price, t - annualised time to expiry
        # r - interest free rate, flag - 'c' or 'p'

        #input interest free rate as fixed
        Interest_free_rate = 0.1

        # Convert the column of Opt_Type for PE as p and CE as c
        df_merged['Opt_Type'] = df_merged['Opt_Type'].apply(lambda x: 'c' if x == 'CE' else 'p')

        #Get only date from timestamp of date for combining
        df_merged['Date_1'] = df_merged['Date'].dt.date

        #Get time value from the timestamp od time for combining
        df_merged['Time'] = pd.to_datetime(df_merged['Time']).dt.time

        #Combine the 2 above
        df_merged['DateTime'] = df_merged.apply(lambda x: datetime.combine(x['Date_1'], x['Time']), axis = 1)

        #Now try to make a similar comibined column for expiry date
        df_merged['Expiry_date_1'] = pd.to_datetime(df_merged['Expiry_date']).dt.date

        # Since end time is fixed to 15:30:00, we will take that as last time of day
        df_merged['ExpiryDateTime'] = df_merged.apply(lambda x : datetime.combine(x['Expiry_date_1'], time(15,30,0)), axis = 1)

        #Calculate years to expiry
        df_merged['Years_to_expiry'] = df_merged.apply(lambda x: abs((x['ExpiryDateTime'] - x['DateTime']).days +
                                                          (x['ExpiryDateTime'] - x['DateTime']).seconds/86400)/365, axis = 1)

        # Dropping unnecesasry columns
        df_merged.drop(columns=['Date', 'Time','Expiry_date','Date_1','Expiry_date_1'],inplace=True)


        #Get IV
        df_merged['IV'] = py_vollib_vectorized.vectorized_implied_volatility(price= df_merged['Close'],
                                                                            S = df_merged['only_nifty_close'],
                                                                            K = df_merged['Strike'],
                                                                            t = df_merged['Years_to_expiry'],
                                                                            r = Interest_free_rate,
                                                                            flag = df_merged['Opt_Type'],
                                                                            q = 0,
                                                                            model = 'black_scholes_merton',
                                                                            return_as= 'series')

        # Now need to calculate all greeks

        df_greeks = py_vollib_vectorized.api.get_all_greeks(S = df_merged['only_nifty_close'],
                                                K = df_merged['Strike'],
                                                t = df_merged['Years_to_expiry'],
                                                r = Interest_free_rate,
                                                sigma = df_merged['IV'],
                                                flag = df_merged['Opt_Type'],
                                                q = 0,
                                                model = 'black_scholes_merton',
                                                return_as= 'dataframe')


        #Final df with all greeks and IVs
        df_final = pd.merge(df_merged, df_greeks, how = 'inner', left_index= True, right_index= True)

        # The deep ITM options are not handled well and hence we are removing these all by using dropna and also resetting index
        # The drop = True removes older index which is not needed now
        df_final_dropNAs = df_final.dropna().reset_index(drop = True)

        # Write the final without NA dataframe to pickle file to store data for posterity
        df_final_dropNAs.to_pickle(r'D:/Options_combined_with_NiftyValues_and_greeks/Nifty_FNO_' + df_final_dropNAs['DateTime'][0].strftime('%d_%m_%Y'))

